In [2]:
import numpy as np
import matplotlib.pyplot as plt
from numba import njit
import pickle
from RebinningFunctions import binning_function_for_x_number_of_bins

In [5]:
bins = np.linspace(1,11,11)
bins

array([ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10., 11.])

In [9]:
counts = np.linspace(1,20,20).reshape(2,10)
counts

array([[ 1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.],
       [11., 12., 13., 14., 15., 16., 17., 18., 19., 20.]])

In [12]:
f = binning_function_for_x_number_of_bins(5)

In [13]:
f(bins, counts, (None,None))

(array([ 1.,  2.,  3.,  4.,  7., 11.]),
 array([[ 1.,  2.,  3., 15., 34.],
        [11., 12., 13., 45., 74.]]))

In [14]:
np.geomspace(bins[0],bins[-1], 6)

array([ 1.        ,  1.61539427,  2.60949864,  4.21536913,  6.80948313,
       11.        ])